In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [57]:
data=pd.read_csv("apart data.CSV", encoding= "EUC-KR")

# 계약날짜를 datetime 형식으로 변환
data['계약날짜'] = pd.to_datetime(data['계약년월'].astype(str) + data['계약일'].astype(str), format='%Y%m%d')

# 입력 변수(X)와 타겟 변수(y) 설정
X = data[['전용면적(㎡)', '층', '건물나이']]
y = data['거래금액(만원)']

In [9]:
data.head(5)

,행정명,번지,본번,부번,단지명,단지명길이,전용면적(㎡),계약년월,계약년,계약월,계약일,거래금액(만원),층,건축년도,건물나이,도로명,해제사유발생일,거래유형,중개사소재지,계약날짜
0,광산구 도산동,1138-2,1138,2,대덕1,3,59.3100,202303,2023,3,18,9800,4,1990,33,송도로 143,NaN,중개거래,광주 광산구,2023-03-18
1,광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202302,2023,2,7,21000,2,2000,23,송도로182번길 55,NaN,중개거래,광주 광산구,2023-02-07
2,광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202305,2023,5,31,22350,5,2000,23,송도로182번길 55,NaN,중개거래,광주 광산구,2023-05-31
3,광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,9,22400,5,2001,22,송도로182번길 16-3,NaN,직거래,-,2023-05-09
4,광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,13,22000,13,2001,22,송도로182번길 16-3,NaN,중개거래,광주 광산구,2023-05-13


In [53]:
len(data['단지명'].unique())

956

In [58]:
# 행정명 변수를 원핫인코딩하여 더미 변수로 변환
onehot_encoded = pd.get_dummies(data['행정명'])

# X에 원핫인코딩된 더미 변수를 추가하여 새로운 X 데이터프레임 생성
X = pd.concat([X, onehot_encoded], axis=1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [59]:
X_train

,전용면적(㎡),층,건물나이,광산구 도산동,광산구 도천동,광산구 비아동,광산구 산월동,광산구 산정동,광산구 선암동,광산구 소촌동,...,서구 동천동,서구 마륵동,서구 매월동,서구 쌍촌동,서구 양동,서구 유촌동,서구 치평동,서구 풍암동,서구 화정동,남구 주월동
46644,84.9652,3,4,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
329695,71.0589,8,3,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150680,59.9180,8,27,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
398547,59.7900,11,12,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
132801,49.1144,4,4,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,84.9893,7,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
365838,84.9600,25,23,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
131932,84.9530,2,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
146867,84.9600,8,32,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [65]:
# 시계열 모델 학습
model = ARIMA(data.set_index('계약날짜')['거래금액(만원)'], order=(1, 1, 1))  
model_fit = model.fit()
arima_forecast = model_fit.forecast(steps=len(X_test))  # 테스트 데이터 크기만큼 
# 앙상블 모델 학습
rf_model = RandomForestRegressor()  # 랜덤 포레스트 모델의 파라미터는 적절히 설정해야 합니다.
rf_model.fit(X_train, y_train)
rf_forecast = rf_model.predict(X_test)

# 시계열 모델과 앙상블 모델의 예측값을 결합하여 최종 예측값 생성
final_prediction = (arima_forecast * 0.5 + rf_forecast * 0.5)  # 예측값 결합 방식은 가중치를 조절하여 조정 가능

# 모델 평가
mse = mean_squared_error(y_test, final_prediction)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, final_prediction)
print("Mean Squared Error:", mse)
print('Route Mean Squared Error:',rmse)
print("R-squared:", r2)

C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. for

Mean Squared Error: 43985021.55453666
Route Mean Squared Error: 6632.1204417996405
R-squared: 0.7161956981149704


In [4]:
from sklearn.preprocessing import OneHotEncoder
# onehotencoder로 원핫인코딩 내용 학습
ohe = OneHotEncoder()
ohe.fit(data[['행정명']])

OneHotEncoder()

In [61]:
from sklearn.preprocessing import OneHotEncoder
# onehotencoder로 원핫인코딩 내용 학습
ohe = OneHotEncoder()
ohe.fit(data[['행정명']])
# X_predict_data를 생성하여 예측값 출력

X_predict_name=" 남구 봉선동" # predict의 이름 입력

res = ohe.transform([[X_predict_name]])
res.toarray()
df = pd.DataFrame(res.toarray(), columns=ohe.get_feature_names_out(['행정명']))
df.columns =df.columns.str.replace('행정명_', '')
df

C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,광산구 도산동,광산구 도천동,광산구 비아동,광산구 산월동,광산구 산정동,광산구 선암동,광산구 소촌동,광산구 송정동,광산구 수완동,광산구 신가동,...,서구 동천동,서구 마륵동,서구 매월동,서구 쌍촌동,서구 양동,서구 유촌동,서구 치평동,서구 풍암동,서구 화정동,남구 주월동
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
X_predict_data = pd.DataFrame([[84, 10, 5]], columns=['전용면적(㎡)', '층', '건물나이'])
X_predict_onehot = df
X_predict_data = pd.concat([X_predict_data, X_predict_onehot], axis=1)


# 시계열 모델의 예측값을 생성
model = ARIMA(data.set_index('계약날짜')['거래금액(만원)'], order=(1, 1, 1))  
arima_forecast_predict = model_fit.forecast(steps=len(X_predict_data))

# 앙상블 모델의 예측값을 생성
rf_forecast_predict = rf_model.predict(X_predict_data)

# 시계열 모델과 앙상블 모델의 예측값을 결합하여 최종 예측값 생성
prediction_predict = (arima_forecast_predict * 0.5 + rf_forecast_predict * 0.5)

print("내일?:", prediction_predict)

내일?: 425079    12825.856816
dtype: float64


C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. for

In [29]:
arima_forecast_predict

425079    15002.21828
dtype: float64

In [64]:
# 미래 날짜 생성
future_dates = pd.date_range(pd.to_datetime('today'), periods=10, freq='MS') 
 # 5개월치의 미래 날짜 생성 (freq='MS'는 월의 시작일을 의미)

# 미래 날짜에 대한 예측값을 위한 X 데이터 생성
X_future = pd.DataFrame([[ 155.824, 10, 17]], columns=['전용면적(㎡)', '층', '건물나이'])
X_future_onehot = df  # 위에서 생성한 더미 변수 데이터 활용
X_future = pd.concat([X_future, X_future_onehot], axis=1)

# 시계열 모델의 미래 예측값 생성
arima_forecast_future = model_fit.forecast(steps=len(future_dates))

# 앙상블 모델의 미래 예측값 생성
rf_forecast_future = rf_model.predict(X_future)

# 시계열 모델과 앙상블 모델의 미래 예측값을 결합하여 최종 미래 예측값 생성
prediction_future = (arima_forecast_future * 0.5 + rf_forecast_future * 0.5)

# 미래 날짜와 해당 날짜의 예측값을 출력
for date, pred in zip(future_dates, prediction_future):
    print("Date:", date.strftime('%Y-%m-%d'), ", Prediction:", pred)

Date: 2023-09-01 , Prediction: 86211.10914006298
Date: 2023-10-01 , Prediction: 86061.73935158047
Date: 2023-11-01 , Prediction: 86043.87443285376
Date: 2023-12-01 , Prediction: 86041.73775364738
Date: 2024-01-01 , Prediction: 86041.482202637
Date: 2024-02-01 , Prediction: 86041.45163823654
Date: 2024-03-01 , Prediction: 86041.4479826745
Date: 2024-04-01 , Prediction: 86041.44754546213
Date: 2024-05-01 , Prediction: 86041.44749317068
Date: 2024-06-01 , Prediction: 86041.44748691651


C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [43]:
future_dates2 = pd.date_range(pd.to_datetime('today'), periods=10, freq='YS')
future_dates2

DatetimeIndex(['2024-01-01 10:26:27.408196', '2025-01-01 10:26:27.408196',
               '2026-01-01 10:26:27.408196', '2027-01-01 10:26:27.408196',
               '2028-01-01 10:26:27.408196', '2029-01-01 10:26:27.408196',
               '2030-01-01 10:26:27.408196', '2031-01-01 10:26:27.408196',
               '2032-01-01 10:26:27.408196', '2033-01-01 10:26:27.408196'],
              dtype='datetime64[ns]', freq='AS-JAN')

In [66]:
feature_names = [f'Feature {i+1}' for i in range(len(feature_importance))]

# 막대 그래프 그리기
plt.figure(figsize=(10, 6))
plt.bar(feature_names, feature_importance)
plt.xticks(rotation=90)  # x축 레이블 회전
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()


NameError: name 'feature_importance' is not defined